In [1]:
%load_ext autoreload
%autoreload 2

import os

import sys
sys.path.append(os.environ['GORDON_REPO_DIR'] + '/pipeline_scripts')

import utilities2014
reload(utilities2014)
from utilities2014 import *

import time

sys.path.append('/home/yuncong/project/opencv-2.4.9/release/lib/python2.7/site-packages')
import cv2

from scipy.spatial.distance import cdist, pdist, squareform
from scipy.cluster.hierarchy import average, fcluster, leaders, complete, single, dendrogram, ward

from joblib import Parallel, delayed

from skimage.color import gray2rgb
from skimage.util import img_as_float, pad
from skimage.morphology import disk
from skimage.filters.rank import gradient

from collections import defaultdict, Counter
from itertools import combinations, chain, product

import networkx
from networkx import from_dict_of_lists, dfs_postorder_nodes

import matplotlib.pyplot as plt
%matplotlib inline

Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
dm = DataManager(generate_hierarchy=False, stack='RS141', resol='x5', section=1)
dm._load_image()

texton_hists = dm.load_pipeline_result('texHist', 'npy')
segmentation = dm.load_pipeline_result('segmentation', 'npy')
n_superpixels = len(np.unique(segmentation)) - 1
textonmap = dm.load_pipeline_result('texMap', 'npy')
n_texton = len(np.unique(textonmap)) - 1
neighbors = dm.load_pipeline_result('neighbors', 'pkl')
sp_properties = dm.load_pipeline_result('spProps', 'npy')
segmentation_vis = dm.load_pipeline_result('segmentationWithText', 'jpg')

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/skimage/filter/__init__.py:6: skimage_deprecation: The `skimage.filter` module has been renamed to `skimage.filters`.  This placeholder module will be removed in v0.13.
  warn(skimage_deprecation('The `skimage.filter` module has been renamed '


In [7]:
dedge_vectors = dm.load_pipeline_result('edgeVectors', 'pkl')
dedge_neighbors = dm.load_pipeline_result('dedgeNeighbors', 'pkl')

In [8]:
edge_coords = dict(dm.load_pipeline_result('edgeCoords', 'pkl'))
all_edges = edge_coords.keys()
all_dedges = set(chain(*[[(i,j),(j,i)] for i,j in all_edges]))

edge_neighbors = dm.load_pipeline_result('edgeNeighbors', 'pkl')

In [10]:
m = dm.load_pipeline_result('boundaryModels', 'pkl')

In [28]:
for i, lm_ind in enumerate([0,4,24,2,17]):

    lm_dedges = [de for de in m[lm_ind][0] if frozenset(de) in edge_coords]

    sample_midpoints = np.array([edge_coords[frozenset(de)].mean(axis=0) for de in lm_dedges], dtype=np.int)
    sample_vectors = np.array([dedge_vectors[de]  for de in lm_dedges])
    sample_int_textures = np.array([texton_hists[de[1]]  for de in lm_dedges])
    sample_ext_textures = np.array([texton_hists[de[0]]  for de in lm_dedges])

    sample_landmark_tuple = np.c_[sample_midpoints, sample_int_textures, sample_ext_textures, sample_vectors]

    np.save('/home/yuncong/csd395/sample_landmark_%d.npy'%i, sample_landmark_tuple)

In [29]:
n_existing_lm = i + 1

In [37]:
i = n_existing_lm
sample_landmark_tuple = np.load('/home/yuncong/csd395/sample_segment.npy')
np.save('/home/yuncong/csd395/sample_landmark_%d.npy'%i, sample_landmark_tuple[::50])

In [36]:
ys = sample_midpoints[::50,1]
xs = sample_midpoints[::50,0]

bg_ubyte = img_as_ubyte(dm.image_rgb)

overlay = np.zeros_like(dm.image, np.bool)
overlay[ys, xs] = 1
overlay = binary_dilation(overlay, disk(5))
bg_ubyte[overlay] = [0,255,0]

display(bg_ubyte)

/oasis/projects/nsf/csd395/yuncong/Brain/notebooks/tmp.jpg

In [19]:
texture_map = texton_hists[segmentation]
texture_map[~dm.mask] = np.nan * np.ones((n_texton,))
dm.save_pipeline_result(texture_map, 'textureMap', 'npy')

saved /oasis/projects/nsf/csd395/yuncong/DavidData2014results/RS141/0001/RS141_x5_0001_gabor-blueNisslWide-segm-blueNisslRegular-vq-blueNissl_textureMap.npy
